In [1]:
from pathlib import Path
from IPython.display import HTML, display
css = Path("../../../css/rtl.css").read_text(encoding="utf-8")
display(HTML(f"<style>{css}</style>"))


# فصل ۱ — مقدمه‌ای بر یادگیری ماشین  
## درس ۹: صورت‌بندی مسئله (ورودی/خروجی، تابع هدف، قیود، هزینه‌ها)

**هدف این درس:** یاد بگیرید چگونه یک سؤال مبهم دنیای واقعی را به یک مشخصات دقیقِ مسئله‌ی یادگیری ماشین تبدیل کنید؛ به‌طوری‌که بتوان آن را پیاده‌سازی کرد، ارزیابی کرد و از نظر حاکمیت/کنترل (governance) تأیید نمود.

در عمل، بسیاری از شکست‌های ML «الگوریتمی» نیستند—بلکه **شکستِ صورت‌بندی** هستند: هدف اشتباه، معیار اشتباه، قیودِ ناگفته، یا هزینه‌های نامنطبق با نیاز کسب‌وکار.


### اهداف یادگیری

در پایان این درس قادر خواهید بود:

1. **ورودی‌ها** $X$ و **خروجی‌ها** $y$ (یا $\hat{y}$) را با واحد، زمان‌بندی و سطحِ تجمیع درست تعریف کنید.
2. یک **تابع هدف** (زیان/منفعت) و یک **پروتکل اندازه‌گیری** (معیارها، برش‌ها، خط‌پایه‌ها) انتخاب کنید.
3. **قیود** را فهرست کنید (داده، تأخیر، حافظه، تفسیرپذیری، انصاف، حریم خصوصی، قوانین، عملیات).
4. **هزینه‌ها** را صریح مدل کنید (مثبت/منفی کاذب، بازبینی انسانی، ریزش، هزینه فرصت).
5. یک «صورت‌بندی یک‌صفحه‌ای» بنویسید که هم تیم فنی بتواند بسازد و هم ذی‌نفع بتواند امضا کند.

برای نشان‌دادن صورت‌بندی‌های مختلف، از چند دیتاست موجود در همین مخزن استفاده می‌کنیم (با مسیرهای نسبی مثل `../../../Datasets/...`).


## ۱. از یک سؤال مبهم تا یک مسئله‌ی ML

سؤال‌های واقعی معمولاً این‌طور مطرح می‌شوند:

- «می‌توانیم پیش‌بینی کنیم کدام مشتری‌ها ریزش می‌کنند؟»
- «می‌توانیم تقلب را تشخیص دهیم؟»
- «می‌توانیم پیشنهادهای بهتری بدهیم؟»
- «می‌توانیم تقاضای ماه آینده را پیش‌بینی کنیم؟»

هر سؤال حداقل چهار انتخاب طراحی را پنهان می‌کند:

1. **ورودی/خروجی**: در *لحظه‌ی تصمیم* چه اطلاعاتی داریم؟ دقیقاً چه چیزی را پیش‌بینی می‌کنیم؟
2. **تابع هدف**: چه چیزی را بهینه می‌کنیم؟ دقت؟ سود؟ ریسک؟ یک جانشین برای خروجی کسب‌وکار؟
3. **قیود**: چه چیزهایی باید رعایت شوند (تأخیر، محاسبات، توضیح‌پذیری، مقررات، انصاف، …)؟
4. **هزینه‌ها**: جریمه‌ی خطا چیست و آیا این جریمه متقارن است؟

یک مدل ذهنی مفید:

$$
\text{Choose } f \in \mathcal{F} \text{ to minimize } \mathbb{E}[L(y, f(X))] \text{ subject to constraints}
$$

که در آن:

- $X$ اطلاعاتی است که هنگام فراخوانی مدل در اختیار داریم،
- $y$ نتیجه‌ای است که واقعاً برای ما مهم است،
- $f$ مدل ماست،
- $L$ تابع زیان (یا منفیِ منفعت) است،
- و قیود «امکان‌پذیری» را مشخص می‌کنند (فنی + سیاستی + کسب‌وکاری).

در ادامه این قطعات را با مثال‌های کدنویسی و مطالعه موردی ملموس می‌کنیم.


In [2]:
import pandas as pd
import numpy as np
import math
from io import StringIO
from pathlib import Path
from IPython.display import display

def load_csv_with_fallback(path, sample_csv_text, synth_rows=500, random_state=42):
    """Load a CSV if it exists; otherwise, create synthetic data with the same columns."""
    path = Path(path)
    if path.exists():
        df = pd.read_csv(path, low_memory=False)
        source = f"Loaded real file: {path}"
        return df, source

    sample_df = pd.read_csv(StringIO(sample_csv_text.strip()))
    cols = list(sample_df.columns)
    rng = np.random.default_rng(random_state)

    df = pd.DataFrame()
    for c in cols:
        if sample_df[c].dtype == object:
            uniq = [u for u in sample_df[c].dropna().unique().tolist() if str(u).strip() != ""]
            if len(uniq) == 0:
                uniq = ["A", "B", "C"]
            df[c] = rng.choice(uniq, size=synth_rows, replace=True)
        else:
            vals = sample_df[c].dropna().to_numpy()
            mu = float(np.mean(vals)) if len(vals) else 0.0
            sigma = float(np.std(vals)) if len(vals) else 1.0
            sigma = sigma if sigma > 1e-9 else 1.0
            df[c] = rng.normal(mu, sigma, size=synth_rows)

    # Make some targets mildly learnable (for demo outputs)
    for target_name in ["classification", "quality", "Drug", "Type", "price", "Price", "magnitude"]:
        if target_name in df.columns:
            if df[target_name].dtype == object:
                num_cols = [c for c in df.columns if df[c].dtype != object and c != target_name]
                if num_cols:
                    z = df[num_cols[0]]
                    uniq = df[target_name].unique().tolist()
                    if len(uniq) < 2:
                        uniq = ["0", "1"]
                    df[target_name] = np.where(z > np.median(z), uniq[0], uniq[1])
            else:
                num_cols = [c for c in df.columns if df[c].dtype != object and c != target_name]
                if num_cols:
                    if len(num_cols) > 1:
                        df[target_name] = 0.7*df[num_cols[0]] + 0.3*df[num_cols[1]] + rng.normal(0, 0.5, size=synth_rows)
                    else:
                        df[target_name] = df[num_cols[0]] + rng.normal(0, 0.5, size=synth_rows)

    source = f"Fallback synthetic data (file not found at: {path})"
    return df, source

def show_basic_profile(df, n=5):
    display(df.head(n))
    print("\nshape:", df.shape)
    print("\ndtypes:\n", df.dtypes)
    print("\nmissing values (top 10):\n", df.isna().sum().sort_values(ascending=False).head(10))


## ۲. ورودی و خروجی: تعریف دقیقِ $X$ و $y$

صورت‌بندی زمانی «قابل اجرا» است که $X$ و $y$ با **زمان‌بندی** و **سطح تجمیع** مشخص شوند.

### زمان‌بندی

اگر نتیجه را با اطلاعاتی پیش‌بینی کنید که **بعد از وقوع نتیجه** در دسترس می‌شود، دچار **نشت داده** شده‌اید.

یک بیان دقیق: $X_t$ ویژگی‌های در دسترس در زمان $t$ (لحظه‌ی تصمیم) و $y_{t+\Delta}$ نتیجه در افق $\Delta$ پس از $t$.

### سطح تجمیع

واحد پیش‌بینی باید شفاف باشد (به ازای فرد، تراکنش، روز-فروشگاه، و ...).

### نوع خروجی

- طبقه‌بندی: $y$ محدود و گسسته
- رگرسیون: $y$ حقیقی
- رتبه‌بندی: خروجی ترتیب
- خوشه‌بندی: $y$ نظارت‌شده نداریم؛ ساختار کشف می‌شود


### مثال A (طبقه‌بندی): ریسک دیابت

`diabetes.csv` را بارگذاری می‌کنیم و یک خط‌پایه می‌سازیم.


In [3]:
diabetes_path = "../../../Datasets/Classification/diabetes.csv"
diabetes_sample = '''Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,classification
6,148,72,35,0,33.6,0.627,50,Diabetic
1,85,66,29,0,26.6,0.351,31,Non-Diabetic
8,183,64,0,0,23.3,0.672,32,Diabetic
1,89,66,23,94,28.1,0.167,21,Non-Diabetic
0,137,40,35,168,43.1,2.288,33,Diabetic
'''
df_diabetes, src_diabetes = load_csv_with_fallback(diabetes_path, diabetes_sample, synth_rows=800)
print(src_diabetes)
show_basic_profile(df_diabetes)


Loaded real file: ..\..\..\Datasets\Classification\diabetes.csv


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,classification
0,6,148,72,35,0,33.6,0.627,50,Diabetic
1,1,85,66,29,0,26.6,0.351,31,Non-Diabetic
2,8,183,64,0,0,23.3,0.672,32,Diabetic
3,1,89,66,23,94,28.1,0.167,21,Non-Diabetic
4,0,137,40,35,168,43.1,2.288,33,Diabetic



shape: (768, 9)

dtypes:
 Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
classification               object
dtype: object

missing values (top 10):
 Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
classification              0
dtype: int64


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, classification_report, brier_score_loss

target_col = "classification"
X_diabetes = df_diabetes.drop(columns=[target_col])
y_diabetes = df_diabetes[target_col].astype(str)

num_cols = [c for c in X_diabetes.columns if X_diabetes[c].dtype != object]
cat_cols = [c for c in X_diabetes.columns if X_diabetes[c].dtype == object]

preprocess = ColumnTransformer([
    ("num", "passthrough", num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
])

clf = Pipeline([
    ("preprocess", preprocess),
    ("model", LogisticRegression(max_iter=2000))
])

X_train, X_test, y_train, y_test = train_test_split(X_diabetes, y_diabetes, test_size=0.25, random_state=42, stratify=y_diabetes)
clf.fit(X_train, y_train)

classes = clf.named_steps["model"].classes_
pos_class = classes[1]
proba_pos = clf.predict_proba(X_test)[:, 1]

y_test_bin = (y_test == pos_class).astype(int)
y_pred = clf.predict(X_test)
y_pred_bin = (y_pred == pos_class).astype(int)

print("Classes:", classes)
print("Positive class:", pos_class)
print("Accuracy:", f"{accuracy_score(y_test, y_pred):.3f}")
print("Precision:", f"{precision_score(y_test_bin, y_pred_bin, zero_division=0):.3f}")
print("Recall:", f"{recall_score(y_test_bin, y_pred_bin, zero_division=0):.3f}")
print("ROC-AUC:", f"{roc_auc_score(y_test_bin, proba_pos):.3f}")
print("Brier:", f"{brier_score_loss(y_test_bin, proba_pos):.3f}")
print("\nConfusion matrix:\n", confusion_matrix(y_test_bin, y_pred_bin))
print("\nReport:\n", classification_report(y_test, y_pred))


Classes: ['Diabetic' 'Non-Diabetic']
Positive class: Non-Diabetic
Accuracy: 0.781
Precision: 0.799
Recall: 0.888
ROC-AUC: 0.825
Brier: 0.161

Confusion matrix:
 [[ 39  28]
 [ 14 111]]

Report:
               precision    recall  f1-score   support

    Diabetic       0.74      0.58      0.65        67
Non-Diabetic       0.80      0.89      0.84       125

    accuracy                           0.78       192
   macro avg       0.77      0.74      0.75       192
weighted avg       0.78      0.78      0.77       192



## ۳. اهداف، معیارها و آستانه تصمیم

آموزش یک زیان مشتق‌پذیر را کمینه می‌کند، اما در استقرار به آستانه $\tau$ برای تصمیم نیاز دارید.

آستانه‌ی آگاه از هزینه‌ها معمولاً بهتر از $\tau=0.5$ است.


In [5]:
from sklearn.metrics import roc_curve
import numpy as np

fpr, tpr, thresholds = roc_curve(y_test_bin, proba_pos)
max_fpr = 0.10
feasible = np.where(fpr <= max_fpr)[0]
best_idx = feasible[np.argmax(tpr[feasible])] if len(feasible) else int(np.argmax(tpr - fpr))
thr = float(thresholds[best_idx])
print(f"Threshold under constraint FPR<=0.10: {thr:.3f}")
print(f"At threshold: FPR={float(fpr[best_idx]):.3f}, TPR={float(tpr[best_idx]):.3f}")


Threshold under constraint FPR<=0.10: 0.812
At threshold: FPR=0.090, TPR=0.544


## ۴. هزینه‌ها: سازش‌های صریح

هزینه‌ی مورد انتظار: $C_{FP}\cdot FP(\tau) + C_{FN}\cdot FN(\tau)$ و انتخاب $\tau$ برای کمینه‌سازی آن.


In [6]:
from sklearn.metrics import confusion_matrix

def expected_cost(cm, c_fp=1.0, c_fn=8.0):
    tn, fp, fn, tp = cm.ravel()
    return c_fp*fp + c_fn*fn

grid = np.linspace(0.01, 0.99, 99)
costs = []
for t in grid:
    pred = (proba_pos >= t).astype(int)
    cm_t = confusion_matrix(y_test_bin, pred)
    costs.append(expected_cost(cm_t, c_fp=1.0, c_fn=8.0))

best = int(np.argmin(costs))
print("Best threshold by expected cost:", float(grid[best]))
print("Min expected cost:", float(costs[best]))


Best threshold by expected cost: 0.08
Min expected cost: 64.0


## ۵. صورت‌بندی رگرسیون (قیمت خانه)

قیمت را مدل می‌کنیم و هزینه خطای متقارن و نامتقارن را مقایسه می‌کنیم.


In [7]:
house_path = "../../../Datasets/Regression/house-prices.csv"
house_sample = '''Home,Price,SqFt,Bedrooms,Bathrooms,Offers,Brick,Neighborhood
1,114300,1790,2,2,2,No,East
2,114200,2030,4,2,3,No,East
3,114800,1740,3,2,1,No,East
4,94700,1980,3,2,3,No,East
5,119800,2130,3,3,3,No,East
'''
df_house, src_house = load_csv_with_fallback(house_path, house_sample, synth_rows=1000)
print(src_house)
show_basic_profile(df_house)


Loaded real file: ..\..\..\Datasets\Regression\house-prices.csv


,Home,Price,SqFt,Bedrooms,Bathrooms,Offers,Brick,Neighborhood
0,1,114300,1790,2,2,2,No,East
1,2,114200,2030,4,2,3,No,East
2,3,114800,1740,3,2,1,No,East
3,4,94700,1980,3,2,3,No,East
4,5,119800,2130,3,3,3,No,East



shape: (128, 8)

dtypes:
 Home             int64
Price            int64
SqFt             int64
Bedrooms         int64
Bathrooms        int64
Offers           int64
Brick           object
Neighborhood    object
dtype: object

missing values (top 10):
 Home            0
Price           0
SqFt            0
Bedrooms        0
Bathrooms       0
Offers          0
Brick           0
Neighborhood    0
dtype: int64


In [8]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

target = "Price" if "Price" in df_house.columns else df_house.columns[1]
X = df_house.drop(columns=[target])
y = df_house[target].astype(float)

num_cols = [c for c in X.columns if X[c].dtype != object]
cat_cols = [c for c in X.columns if X[c].dtype == object]

pre = ColumnTransformer([
    ("num", "passthrough", num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
])

reg = Pipeline([("pre", pre), ("model", Ridge(alpha=1.0))])
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.25, random_state=42)
reg.fit(Xtr, ytr)
pred = reg.predict(Xte)

print("R^2:", f"{r2_score(yte, pred):.3f}")
print("MAE:", f"{mean_absolute_error(yte, pred):.2f}")
print("RMSE:", f"{math.sqrt(mean_squared_error(yte, pred)):.2f}")


R^2: 0.824
MAE: 8400.34
RMSE: 10373.56


In [9]:
import numpy as np

def asym_loss(y_true, y_pred, alpha=5.0, beta=1.0):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    over = np.maximum(y_pred - y_true, 0.0)
    under = np.maximum(y_true - y_pred, 0.0)
    return np.mean(alpha*over + beta*under)

for a,b in [(1,1),(2,1),(5,1),(1,2)]:
    print(f"alpha={a}, beta={b} -> loss={asym_loss(yte, pred, alpha=a, beta=b):.2f}")


alpha=1, beta=1 -> loss=8400.34
alpha=2, beta=1 -> loss=12226.91
alpha=5, beta=1 -> loss=23706.59
alpha=1, beta=2 -> loss=12974.12


## ۶. صورت‌بندی بدون‌نظارت (خوشه‌بندی شکایت‌ها)

اینجا $y$ نداریم؛ هدف کشف تعریف می‌کنید و خوشه‌ها را با بازبینی انسانی اعتبارسنجی می‌کنید.


In [10]:
comp_path = "../../../Datasets/Clustering/ConsumerComplaints.csv"
comp_sample = '''Date Received,Product Name,Sub Product,Issue,Sub Issue,Consumer Complaint Narrative,Company Public Response,Company,State Name,Zip Code,Tags,Consumer Consent Provided,Submitted via,Date Sent to Company,Company Response to Consumer,Timely Response,Consumer Disputed,Complaint ID
2013-07-29,Consumer Loan,Vehicle loan,Managing the loan or lease,,,,Wells Fargo & Company,VA,24540,,N/A,Phone,2013-07-30,Closed with explanation,Yes,No,468882
2013-07-29,Bank account or service,Checking account,Using a debit or ATM card,,,,Wells Fargo & Company,CA,95992,Older American,N/A,Web,2013-07-31,Closed with explanation,Yes,No,468889
2013-07-29,Bank account or service,Checking account,"Account opening, closing, or management",,,,Santander Bank US,NY,10065,,N/A,Fax,2013-07-31,Closed,Yes,No,468879
2013-07-29,Bank account or service,Checking account,Deposits and withdrawals,,,,Wells Fargo & Company,GA,30084,,N/A,Web,2013-07-30,Closed with explanation,Yes,No,468949
2013-07-29,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",,,,Franklin Credit Management,CT,6106,,N/A,Web,2013-07-30,Closed with explanation,Yes,No,475823
'''
df_comp, src_comp = load_csv_with_fallback(comp_path, comp_sample, synth_rows=1200)
print(src_comp)
show_basic_profile(df_comp, n=3)


Loaded real file: ..\..\..\Datasets\Clustering\ConsumerComplaints.csv


,Date Received,Product Name,Sub Product,Issue,Sub Issue,Consumer Complaint Narrative,Company Public Response,Company,State Name,Zip Code,Tags,Consumer Consent Provided,Submitted via,Date Sent to Company,Company Response to Consumer,Timely Response,Consumer Disputed,Complaint ID
0,2013-07-29,Consumer Loan,Vehicle loan,Managing the loan or lease,NaN,NaN,NaN,Wells Fargo & Company,VA,24540,NaN,NaN,Phone,2013-07-30,Closed with explanation,Yes,No,468882
1,2013-07-29,Bank account or service,Checking account,Using a debit or ATM card,NaN,NaN,NaN,Wells Fargo & Company,CA,95992,Older American,NaN,Web,2013-07-31,Closed with explanation,Yes,No,468889
2,2013-07-29,Bank account or service,Checking account,"Account opening, closing, or management",NaN,NaN,NaN,Santander Bank US,NY,10065,NaN,NaN,Fax,2013-07-31,Closed,Yes,No,468879



shape: (65499, 18)

dtypes:
 Date Received                   object
Product Name                    object
Sub Product                     object
Issue                           object
Sub Issue                       object
Consumer Complaint Narrative    object
Company Public Response         object
Company                         object
State Name                      object
Zip Code                        object
Tags                            object
Consumer Consent Provided       object
Submitted via                   object
Date Sent to Company            object
Company Response to Consumer    object
Timely Response                 object
Consumer Disputed               object
Complaint ID                     int64
dtype: object

missing values (top 10):
 Company Public Response         63049
Consumer Complaint Narrative    62885
Consumer Consent Provided       60739
Tags                            55818
Sub Issue                       34841
Sub Product                     18564

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

text_cols = [c for c in ["Product Name", "Sub Product", "Issue", "Sub Issue", "Company", "State Name"] if c in df_comp.columns]
df_comp["cluster_text"] = df_comp[text_cols].astype(str).agg(" | ".join, axis=1)

vec = TfidfVectorizer(min_df=2, max_features=4000)
X_text = vec.fit_transform(df_comp["cluster_text"])

ks = [3,5,8]
best_k, best_s = None, -1
best_labels, best_model = None, None
for k in ks:
    km = KMeans(n_clusters=k, n_init=10, random_state=42)
    labels = km.fit_predict(X_text)
    s = silhouette_score(X_text, labels, sample_size=min(400, X_text.shape[0]), random_state=42)
    print(f"k={k}, silhouette(sampled)={s:.3f}")
    if s > best_s:
        best_k, best_s = k, s
        best_labels, best_model = labels, km

print("Selected k:", best_k)


k=3, silhouette(sampled)=0.109
k=5, silhouette(sampled)=0.144
k=8, silhouette(sampled)=0.134
Selected k: 5


k=5, silhouette(sampled)=0.283


k=8, silhouette(sampled)=0.207
Selected k: 5


In [12]:
import numpy as np
terms = np.array(vec.get_feature_names_out())
centers = best_model.cluster_centers_
top_idx = centers.argsort(axis=1)[:, -8:][:, ::-1]
for i, row in enumerate(top_idx):
    print(f"Cluster {i}: " + ", ".join(terms[row]))


Cluster 0: credit, information, report, incorrect, on, reporting, status, experian
Cluster 1: debt, not, attempts, owed, cont, collect, collection, mine
Cluster 2: mortgage, loan, foreclosure, modification, conventional, nan, servicing, escrow
Cluster 3: credit, card, nan, loan, or, debt, student, of
Cluster 4: account, bank, service, or, checking, opening, closing, management


## ۷. چک‌لیست قیود که نباید حذف کنید

حتی در تمرین آموزشی، این موارد را بنویسید:

- اهداف تأخیر و توان عبوری
- سیاست دسترسی به داده و حریم خصوصی (PII)
- تحلیل انصاف و آسیب (چه کسانی ممکن است از خطا آسیب ببینند؟)
- برنامه پایش (درفت، افت عملکرد)
- محرک‌های بازآموزی و برنامه بازگشت
- معیارهای انسان-در-حلقه

مدلی که «در نوت‌بوک خوب کار کند» اما قیود را نقض کند، قابل استقرار نیست.


## ۸. جمع‌بندی

صورت‌بندی معیار موفقیت را تعیین می‌کند. قبل از تیون‌کردن مدل‌ها، مطمئن شوید پاسخ این‌ها را دارید:

1. مدل از چه تصمیمی پشتیبانی می‌کند؟
2. هدف و سیاست برچسب چیست؟
3. کدام معیار به هزینه و قیود وصل است؟
4. کدام حالت‌های شکست غیرقابل قبول‌اند؟
5. نقطه عملیاتی (آستانه، top-k، یا سیاست) چیست؟
